In [2]:
# ---------------------------------------------------------
# Task 1: Setup & Text Normalization
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Display settings
pd.set_option('display.max_colwidth', 100)

# 1. Load the Parsed Data
# Ensure the path points to your saved CSV from Phase 1
input_path = Path(r'../data/processed/parsed_data.csv')
df = pd.read_csv(input_path, encoding='utf-8-sig')

print(f"Loaded {len(df)} reviews.")

# ---------------------------------------------------------
# DEFINITION: Persian Text Normalizer
# ---------------------------------------------------------
def normalize_persian(text):
    """
    Standardizes Persian characters:
    1. Converts Arabic 'ي' and 'ك' to Persian 'ی' and 'ک'.
    2. Removes Arabic diacritics (fatha, damma, etc.).
    3. Replaces half-spaces (ZWNJ) with standard spaces (optional, usually better for grouping).
    4. Strips extra whitespace.
    """
    if not isinstance(text, str):
        return text
    
    # Character Translation Map
    translations = str.maketrans({
        'ي': 'ی',
        'ك': 'ک',
        'ة': 'ه',
        'آ': 'ا',
        'إ': 'ا',
        'أ': 'ا',
        'ؤ': 'و',
        '\u200c': ' ',  # Convert Zero-Width Non-Joiner (half-space) to space
    })
    
    text = text.translate(translations)
    
    # Remove Arabic Diacritics (Fatha, Kasra, etc.)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    
    # Collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# ---------------------------------------------------------
# EXECUTION: Normalize Text Columns
# ---------------------------------------------------------

# We apply this to Name, Dept, Course, and Comments
text_cols = ['professor_name_raw', 'department', 'course_name', 'comment_text']

for col in text_cols:
    # Create new 'clean_' columns so we preserve the original data for comparison
    df[f'clean_{col}'] = df[col].apply(normalize_persian)

print("Text Normalization Complete.")

# Show a sample of the changes
display(df[['professor_name_raw', 'clean_professor_name_raw']].head())

Loaded 4548 reviews.
Text Normalization Complete.


,professor_name_raw,clean_professor_name_raw
0,سید محمد شهرتاش,سید محمد شهرتاش
1,دکتر هاجر فلاحتی,دکتر هاجر فلاحتی
2,علم الهدی,علم الهدی
3,حجت قاسمی,حجت قاسمی
4,فریبرز قاسمی,فریبرز قاسمی


In [3]:
# Filter for rows where the cleaning actually changed something
changes = df[df['professor_name_raw'] != df['clean_professor_name_raw']]

print(f"Total rows changed: {len(changes)}")

if len(changes) > 0:
    print("\nSample Differences (Hidden characters revealed):")
    # We display the length to prove they are different strings
    sample = changes[['professor_name_raw', 'clean_professor_name_raw']].head(5).copy()
    sample['Raw Length'] = sample['professor_name_raw'].apply(len)
    sample['Clean Length'] = sample['clean_professor_name_raw'].apply(len)
    display(sample)
else:
    print("No changes found (Data might have been clean already).")

Total rows changed: 246

Sample Differences (Hidden characters revealed):


,professor_name_raw,clean_professor_name_raw,Raw Length,Clean Length
19,ساسان آسیایی,ساسان اسیایی,12,12
26,سید حسن هاشم آبادی,سید حسن هاشم ابادی,18,18
42,مجید ایلچی قزاآن,مجید ایلچی قزاان,16,16
48,اسداله آقاجانی,اسداله اقاجانی,14,14
87,آسیه سادات کاظمی,اسیه سادات کاظمی,16,16


In [7]:
# ---------------------------------------------------------
# NORMALIZATION TEST SUITE (Corrected)
# ---------------------------------------------------------

def run_normalization_tests():
    
    test_cases = [
        # Case 1: Arabic Y and K
        ("علي كریمی", "علی کریمی", "Arabic 'Y' and 'K' conversion"),
        
        # Case 2: ZWNJ -> Space (Standard behavior)
        ("دانشکده\u200cفنی", "دانشکده فنی", "Zero-Width Non-Joiner removal"),
        
        # Case 3: Diacritics
        ("مُحَمّد", "محمد", "Arabic Diacritics stripping"),
        
        # Case 4: Whitespace
        ("   دکتر    رضایی  \n", "دکتر رضایی", "Whitespace cleaning"),
        
        # Case 5: Mixed Edge Case
        # CORRECTED EXPECTATION: ZWNJ becomes a space, matching the output we want
        ("  آقاي مُحَمّد\u200cرضا   كاظمی  ", "اقای محمد رضا کاظمی", "Mixed Edge Case") 
    ]

    passed_count = 0
    
    for i, (raw, expected, desc) in enumerate(test_cases, 1):
        result = normalize_persian(raw)
        
        if result == expected:
            print(f"   TEST {i}: {desc} -> PASS")
            passed_count += 1
        else:
            print(f"   TEST {i}: {desc} -> FAIL")
            print(f"   Input:    '{raw}'")
            print(f"   Expected: '{expected}'")
            print(f"   Got:      '{result}'")
            
    if passed_count == len(test_cases):
        print("\n   ALL SYSTEM TESTS PASSED.")

# Run the corrected tests
run_normalization_tests()

   TEST 1: Arabic 'Y' and 'K' conversion -> PASS
   TEST 2: Zero-Width Non-Joiner removal -> PASS
   TEST 3: Arabic Diacritics stripping -> PASS
   TEST 4: Whitespace cleaning -> PASS
   TEST 5: Mixed Edge Case -> PASS

   ALL SYSTEM TESTS PASSED.


In [8]:
# ---------------------------------------------------------
# DEEP SCAN: Audit the Final Data
# ---------------------------------------------------------
print("\n" + "="*40)
print("   DEEP SCANNING PROCESSED DATA...")

# Define Forbidden Characters (Regex)
forbidden_patterns = {
    "Arabic Y (ي)": r'ي',
    "Arabic K (ك)": r'ك',
    "Arabic Diacritics": r'[\u064B-\u065F]',
    "Zero Width Joiner": r'\u200c',
    "Double Spaces": r'\s{2,}'
}

# Check the 'clean_professor_name_raw' column
target_col = 'clean_professor_name_raw'
scan_failed = False

for name, pattern in forbidden_patterns.items():
    # Find rows that match the forbidden pattern
    bad_rows = df[df[target_col].str.contains(pattern, regex=True, na=False)]
    
    if len(bad_rows) > 0:
        scan_failed = True
        print(f"   FAILURE: Found {len(bad_rows)} rows with {name}")
        print(f"   Sample: {bad_rows[target_col].iloc[0]}")
    else:
        print(f"   PASSED: No '{name}' found.")

if not scan_failed:
    print("\n   CERTIFIED CLEAN: The data contains no forbidden characters.")
else:
    print("\n   ALERT: The data requires further cleaning.")


   DEEP SCANNING PROCESSED DATA...
   PASSED: No 'Arabic Y (ي)' found.
   PASSED: No 'Arabic K (ك)' found.
   PASSED: No 'Arabic Diacritics' found.
   PASSED: No 'Zero Width Joiner' found.
   PASSED: No 'Double Spaces' found.

   CERTIFIED CLEAN: The data contains no forbidden characters.


In [11]:
# ---------------------------------------------------------
# Step 2: Remove Honorifics (Titles)
# ---------------------------------------------------------
import re

def remove_titles(name):
    if not isinstance(name, str): return ""
    
    titles = [
        r'\bدکتر\b',   # Doctor
        r'\bاستاد\b',  # Professor
        r'\bمهندس\b',  # Engineer
        r'\bخانم\b',   # Ms./Mrs.
        r'\bآقای\b',   # Mr.
        r'\bحاج\b'     # Haji (Usually a title, safe to remove if separate word)
    ]
    
    clean_name = name
    for title in titles:
        # Replace title with empty string
        clean_name = re.sub(title, '', clean_name)
    
    return clean_name.strip()

# --- UNIT TESTS (Updated for constraints) ---

# 1. Standard Title Removal
assert remove_titles("دکتر علی رضایی") == "علی رضایی"
assert remove_titles("مهندس محمدی") == "محمدی"

# 2. "Seyed" Preservation Tests (CRITICAL)
assert remove_titles("سید علی حسینی") == "سید علی حسینی"
assert remove_titles("خانم سیده زهرا") == "سیده زهرا"

# 3. Edge Cases
assert remove_titles("استاد سید محمد") == "سید محمد"

print("Title Remover Tests Passed.")

Title Remover Tests Passed.


In [13]:
# ---------------------------------------------------------
# EXECUTION: Apply Corrected Title Removal
# ---------------------------------------------------------

# Create the 'name_no_title' column
df['name_no_title'] = df['clean_professor_name_raw'].apply(remove_titles)

# Show a sample of what actually changed
# This helps us confirm that "Dr. Ali" -> "Ali" happened, but "Seyed Ali" stayed "Seyed Ali"
changed_rows = df[df['clean_professor_name_raw'] != df['name_no_title']]

print(f"Titles removed. {len(changed_rows)} names were simplified.")

if len(changed_rows) > 0:
    print("\nSample Changes:")
    display(changed_rows[['clean_professor_name_raw', 'name_no_title']].drop_duplicates().head(10))

Titles removed. 183 names were simplified.

Sample Changes:


,clean_professor_name_raw,name_no_title
1,دکتر هاجر فلاحتی,هاجر فلاحتی
17,دکتر رامین هاشمی,رامین هاشمی
93,دکتر شاهرخ فرهمند,شاهرخ فرهمند
94,دکتر داود عرب خابوری,داود عرب خابوری
95,دکتر علی صدر,علی صدر
96,دکتر سید محمد شهرتاش,سید محمد شهرتاش
98,دکتر سید ادیب ابریشمی فر,سید ادیب ابریشمی فر
105,دکتر عبادالهی,عبادالهی
144,دکتر الهام فتاحی,الهام فتاحی
367,دکتر رضا احمدی,رضا احمدی


In [16]:
from fuzzywuzzy import process, fuzz
import pandas as pd

# ---------------------------------------------------------
# 1. IMPROVED ALGORITHM: Fuzzy + Subset Logic
# ---------------------------------------------------------
def get_standardized_names(df, similarity_threshold=85):
    name_mapping = {}
    
    # Handle missing dept case for testing
    if 'department' in df.columns:
        departments = df['department'].unique()
    else:
        departments = [None]
    
    for dept in departments:
        # Get names for this group
        if dept is not None:
            dept_names = df[df['department'] == dept]['name_no_title'].unique()
        else:
            dept_names = df['name_no_title'].unique()
        
        # Sort by length (Longest is Standard)
        dept_names = sorted(dept_names, key=len, reverse=True)
        
        processed_in_dept = set()
        
        for name in dept_names:
            if name in processed_in_dept:
                continue
            
            standard = name
            name_mapping[name] = standard
            processed_in_dept.add(name)
            
            # Check remaining candidates
            candidates = [n for n in dept_names if n not in processed_in_dept]
            
            for candidate in candidates:
                # CRITERIA 1: Fuzzy Match (Strict)
                # usage of token_sort_ratio prevents "Ali Mohammadi" merging with "Hassan Mohammadi"
                fuzzy_score = fuzz.token_sort_ratio(standard, candidate)
                
                # CRITERIA 2: Subset Match (The "Rezaei" Fix)
                # Checks if "Rezaei" is strictly inside "Alireza Rezaei" based on words
                std_tokens = set(standard.split())
                cand_tokens = set(candidate.split())
                is_subset = cand_tokens.issubset(std_tokens)
                
                if fuzzy_score >= similarity_threshold or is_subset:
                    name_mapping[candidate] = standard
                    processed_in_dept.add(candidate)
                    
    return name_mapping

# ---------------------------------------------------------
# 2. VERIFICATION: Run the Test Suite Again
# ---------------------------------------------------------
def run_fuzzy_tests():
    
    test_df = pd.DataFrame({
        'department': ['Math', 'Math', 'Physics', 'Physics', 'Arts', 'Arts'],
        'name_no_title': [
            'علیرضا رضایی', 'رضایی',         # Case 1: Substring (Subset Logic)
            'علی محمدی', 'حسن محمدی',        # Case 2: Distinct IDs (Fuzzy Logic Safety)
            'احمدی نژاد', 'نژاد احمدی'       # Case 3: Reordered (Fuzzy Logic Flexibility)
        ]
    })
    
    mapping = get_standardized_names(test_df, similarity_threshold=85)
    
    # TEST 1: The one that failed before
    if mapping.get('رضایی') == 'علیرضا رضایی':
        print("TEST 1: Subset Match (Rezaei -> Alireza Rezaei) -> PASS")
    else:
        print(f"TEST 1 FAIL: Got {mapping.get('رضایی')}")

    # TEST 2: The safety check
    if mapping.get('حسن محمدی') == 'حسن محمدی':
        print("TEST 2: Distinct People Safety (Ali vs Hasan) -> PASS")
    else:
        print(f"TEST 2 FAIL: Merged incorrectly!")

    # TEST 3: Order check
    if mapping.get('نژاد احمدی') == 'احمدی نژاد':
        print("TEST 3: Word Order Match -> PASS")
    else:
        print(f"TEST 3 FAIL: Order check failed")

    print("\nALL LOGIC CHECKS PASSED.")

run_fuzzy_tests()

TEST 1: Subset Match (Rezaei -> Alireza Rezaei) -> PASS
TEST 2: Distinct People Safety (Ali vs Hasan) -> PASS
TEST 3: Word Order Match -> PASS

ALL LOGIC CHECKS PASSED.


In [23]:
import pandas as pd
from fuzzywuzzy import fuzz

def check_risk_pairs_pretty():
    print("⚠️ DANGER ZONE TESTING (THRESHOLD = 85)...\n")
    
    pairs = [
        # Case 1: Ali vs Alireza (Should NOT merge)
        ("علیرضا رضایی", "علی رضایی"),
        
        # Case 2: Reza vs Alireza (Should NOT merge)
        ("علیرضا رضایی", "رضا رضایی"),
        
        # Case 3: Same First Name, Diff Last Name (Should NOT merge)
        ("علی محمدی", "علی احمدی"),
        
        # Case 4: The Valid Merge (Rezaei -> Alireza Rezaei)
        ("علیرضا رضایی", "رضایی")
    ]
    
    threshold = 85
    results = []
    
    for name_a, name_b in pairs:
        # Calculate strict fuzzy score
        score = fuzz.token_sort_ratio(name_a, name_b)
        
        # Calculate Subset Logic
        tokens_a = set(name_a.split())
        tokens_b = set(name_b.split())
        is_subset = tokens_b.issubset(tokens_a)
        
        # Decision Logic
        if is_subset:
            action = "MERGE (Subset)"
        elif score >= threshold:
            action = "MERGE (Fuzzy)" # <--- Risk Factor
        else:
            action = "KEEP SEPARATE"
            
        results.append({
            'Name A (Standard)': name_a,
            'Name B (Candidate)': name_b,
            'Score': score,
            'Is Subset?': is_subset,
            'Action': action
        })
    
    # Return pretty table
    return pd.DataFrame(results)

# Run and display
display(check_risk_pairs_pretty())

⚠️ DANGER ZONE TESTING (THRESHOLD = 85)...



,Name A (Standard),Name B (Candidate),Score,Is Subset?,Action
0,علیرضا رضایی,علی رضایی,86,False,MERGE (Fuzzy)
1,علیرضا رضایی,رضا رضایی,67,False,KEEP SEPARATE
2,علی محمدی,علی احمدی,44,False,KEEP SEPARATE
3,علیرضا رضایی,رضایی,59,True,MERGE (Subset)


In [22]:
# ---------------------------------------------------------
# DANGER ZONE TEST (Clean Output)
# ---------------------------------------------------------
def check_risk_pairs_pretty():
    print("🛡️ DANGER ZONE TESTING (THRESHOLD = 90)...\n")
    
    pairs = [
        ("علیرضا رضایی", "علی رضایی"),  # The problem case
        ("علیرضا رضایی", "رضا رضایی"),
        ("علی محمدی", "علی احمدی"),
        ("علیرضا رضایی", "رضایی")       # The subset case
    ]
    
    THRESHOLD = 90
    results = []
    
    for name_a, name_b in pairs:
        score = fuzz.token_sort_ratio(name_a, name_b)
        
        # Calculate Subset Logic
        tokens_a = set(name_a.split())
        tokens_b = set(name_b.split())
        is_subset = tokens_b.issubset(tokens_a)
        
        if is_subset:
            action = "MERGE (Subset)"
        elif score >= THRESHOLD:
            action = "MERGE (Fuzzy)"
        else:
            action = "KEEP SEPARATE"
            
        results.append({
            'Name A (Standard)': name_a,
            'Name B (Candidate)': name_b,
            'Score': score,
            'Is Subset?': is_subset,
            'Final Action': action
        })
    
    # Return as a Pandas DataFrame for nice formatting
    return pd.DataFrame(results)

# Run and display
display(check_risk_pairs_pretty())

🛡️ DANGER ZONE TESTING (THRESHOLD = 90)...



,Name A (Standard),Name B (Candidate),Score,Is Subset?,Final Action
0,علیرضا رضایی,علی رضایی,86,False,KEEP SEPARATE
1,علیرضا رضایی,رضا رضایی,67,False,KEEP SEPARATE
2,علی محمدی,علی احمدی,44,False,KEEP SEPARATE
3,علیرضا رضایی,رضایی,59,True,MERGE (Subset)


In [24]:
import pandas as pd
from fuzzywuzzy import fuzz

def check_ahmad_mohammad_risk():
    
    pairs = [
        # The Case You Feared: Same Last Name, Similar First Names
        ("احمد رضایی", "محمد رضایی"),
        
        # Another Common Mix-up: Hamid vs Mohammad
        ("حمید رضایی", "محمد رضایی"),
        
        # A Real Typo (Should still merge)
        ("علیرضا رضایی", "علیرضا رضائی") 
    ]
    
    # Let's test against strict thresholds
    thresholds = [90, 95, 96]
    results = []
    
    for name_a, name_b in pairs:
        score = fuzz.token_sort_ratio(name_a, name_b)
        
        # Subset logic (always active)
        is_subset = set(name_b.split()).issubset(set(name_a.split()))
        
        for thresh in thresholds:
            if is_subset:
                action = "MERGE (Subset)"
            elif score >= thresh:
                action = f"MERGE (Risk @ {thresh})" 
            else:
                action = "KEEP SEPARATE"
            
            results.append({
                'Name Pair': f"{name_a} vs {name_b}",
                'Score': score,
                'Threshold': thresh,
                'Action': action
            })
            
    return pd.pivot_table(pd.DataFrame(results), values='Action', index=['Name Pair', 'Score'], columns=['Threshold'], aggfunc='first')

display(check_ahmad_mohammad_risk())

,Threshold,90,95,96
Name Pair,Score,,,
احمد رضایی vs محمد رضایی,50,KEEP SEPARATE,KEEP SEPARATE,KEEP SEPARATE
حمید رضایی vs محمد رضایی,50,KEEP SEPARATE,KEEP SEPARATE,KEEP SEPARATE
علیرضا رضایی vs علیرضا رضائی,92,MERGE (Risk @ 90),KEEP SEPARATE,KEEP SEPARATE


In [25]:
# ---------------------------------------------------------
# EXECUTION: FINAL STANDARDIZATION (Threshold 90)
# ---------------------------------------------------------

# We use 90 because it filters distinct people (Score ~86) 
# but catches valid typos like Hamza (Score ~92)
name_map = get_standardized_names(df, similarity_threshold=90)

# Apply mapping
df['professor_name_standard'] = df['name_no_title'].map(name_map).fillna(df['name_no_title'])

# Stats
n_before = df['clean_professor_name_raw'].nunique()
n_after = df['professor_name_standard'].nunique()

print(f"Standardization Complete.")
print(f"Total Unique Names Before: {n_before}")
print(f"Total Unique Names After:  {n_after}")
print(f"Reduction: {n_before - n_after} duplicates merged.")

# Show verified merges
merged = df[df['name_no_title'] != df['professor_name_standard']]
if len(merged) > 0:
    print("\nFinal Verified Merges:")
    display(merged[['department', 'name_no_title', 'professor_name_standard']].drop_duplicates().head(15))

Standardization Complete.
Total Unique Names Before: 1269
Total Unique Names After:  782
Reduction: 487 duplicates merged.

Final Verified Merges:


,department,name_no_title,professor_name_standard
2,مهندسی مواد,علم الهدی,سمیه علم الهدی
4,مهندسی کامپیوتر,فریبرز قاسمی,فریبرز قاسمی فیض ابادی
8,برق,سید علیرضا نظام الحسینی,سید علیرضا نظام الحسینی عز ابادی
11,فیزیک,محمدرضا زمانی,محمدرضا زمانی میمیان
15,مکانیک,حمید صفاری,حمید صفاری نطنزی
20,ریاضی,علی پارسا,علی پارسا ریاضی دو
22,ریاضی,محبوبه مولوی عربشاهی,سیده محبوبه مولوی عربشاهی
38,ریاضی,احمد فروغ,احمد رضا فروغ
40,معارف,حسین سجادی,سید حسین سجادی
48,ریاضی,اسداله اقاجانی,اسدالله اقاجانی


In [27]:
import re

# ---------------------------------------------------------
#            Robust Course Name Unification
# ---------------------------------------------------------
def normalize_course_names(text):
    if not isinstance(text, str): return ""
    
    # 1. Basic Persian Normalization (your existing function)
    text = normalize_persian(text)
    
    # 2. Digit Conversion (Persian/Arabic -> English)
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    arabic_digits = "٠١٢٣٤٥٦٧٨٩"
    english_digits = "0123456789"
    translation_table = str.maketrans(persian_digits + arabic_digits, english_digits * 2)
    text = text.translate(translation_table)
    
    # 3. ROBUSTNESS: Remove common junk punctuation
    # Replaces ( ) - _ with a space
    text = re.sub(r'[()\-–_]', ' ', text)
    
    # 4. ROBUSTNESS: Fix "Glued" Numbers (e.g., "Math1" -> "Math 1")
    # Adds a space if a letter is immediately followed by a number
    text = re.sub(r'([a-zA-Z\u0600-\u06FF])(\d)', r'\1 \2', text)
    
    # 5. Final Cleanup (Remove double spaces created by step 3 & 4)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply to the 'course_name' column (Check your column name!)
target_col = 'course_name'  # <--- Verify this name matches your dataframe

if target_col in df.columns:
    df['clean_course_name'] = df[target_col].apply(normalize_course_names)
    
    n_before = df[target_col].nunique()
    n_after = df['clean_course_name'].nunique()
    
    print(f"Course Names Unified.")
    print(f"Unique Courses Before: {n_before}")
    print(f"Unique Courses After:  {n_after}")
    print(f"Reduction: {n_before - n_after} variations merged.")
    
    # SHOW THE PROOF: List top 10 most frequent courses to spot check
    print("\nTop 10 Most Common Courses (After Cleaning):")
    print(df['clean_course_name'].value_counts().head(10))

else:
    print(f"Error: Column '{target_col}' not found.")

Course Names Unified.
Unique Courses Before: 1330
Unique Courses After:  1079
Reduction: 251 variations merged.

Top 10 Most Common Courses (After Cleaning):
clean_course_name
معادلات دیفرانسیل    212
ریاضی 1              176
ریاضی 2              158
فیزیک 2              135
فیزیک 1              124
استاتیک               77
فارسی                 77
اندیشه 1              75
اخلاق اسلامی          70
تربیت بدنی            61
Name: count, dtype: int64


In [30]:
# ---------------------------------------------------------
# INSPECTION: View Most Common Descriptions (Corrected Columns)
# ---------------------------------------------------------

# Correct Column Names found in your dataframe
GRADING_COL = 'grading_status_raw'
ATTENDANCE_COL = 'attendance_status_raw'

print(f"Top 30 Most Frequent 'Grading' Comments:")
if GRADING_COL in df.columns:
    # We display the index (the text) and the count
    print(df[GRADING_COL].value_counts().head(30))
else:
    print(f"Error: Still can't find '{GRADING_COL}'")

print("\n" + "="*40 + "\n")

print(f"Top 30 Most Frequent 'Attendance' Comments:")
if ATTENDANCE_COL in df.columns:
    print(df[ATTENDANCE_COL].value_counts().head(30))
else:
    print(f"Error: Still can't find '{ATTENDANCE_COL}'")

Top 30 Most Frequent 'Grading' Comments:
grading_status_raw
منصفانه و هرچی خودت بگیری                                                        1682
دست باز و با ارفاق                                                               1271
نمره خوبی نمیشه ازشون گرفت                                                       1166
یادم نیست چجوری بود                                                                44
رندوم                                                                               4
تا حدودی                                                                            2
افتضاح                                                                              2
رندوم نمره میدن                                                                     2
بسیارمنصفانه ومهربانانه                                                             2
خیر                                                                                 2
خب اساسا نمره ای که میدن طوریه که سخت گیره من خودم حس کردم نمرم باید بیشتر شه   

In [32]:
# ---------------------------------------------------------
# TASK 4, PART 2: FINAL CLASSIFIER (Re-run Safe)
# ---------------------------------------------------------

# We read from RAW columns (Safe to re-run this cell many times)
GRADING_COL = 'grading_status_raw'
ATTENDANCE_COL = 'attendance_status_raw'

def categorize_grading(text):
    if not isinstance(text, str): return "نامشخص"
    text = text.lower()
    
    # 0. Unknowns
    if any(k in text for k in ['یادم نیست', 'نمیدونم', 'نیومده', 'خیر', 'ندارم']):
        return "نامشخص"

    # 1. Easy (آسان)
    if any(k in text for k in ['دست باز', 'ارفاق', 'خوب نمره', 'عالی', '۲۰', '20', 'هلو', 'خوش نمره', 'مهربان', 'مطلوب']):
        return "آسان"
    
    # 2. Strict / Negative (سخت‌گیر)
    # CRITICAL FIX: We check 'غیر منصفانه' and 'نه چندان' HERE so they don't get trapped as 'Fair' later.
    strict_keywords = [
        'نمیشه', 'نمی ده', 'نمیده', 'سخت', 'رندوم', 'شانسی', 'افتضاح', 
        'کمتر', 'بد', 'شانس', 'کشکی', 'غیر منطقی', 'ده بیست', 
        'نه چندان', 'غیر منصفانه', 'صحیحی ندارند', 'پایین'
    ]
    if any(k in text for k in strict_keywords):
        return "سخت‌گیر"
    
    # 3. Fair (منصفانه)
    # Only runs if the negative checks above failed
    if any(k in text for k in ['منصفانه', 'حق', 'هرچی', 'واقعی', 'نرمال']):
        return "منصفانه"
        
    return "نامشخص"

def categorize_attendance(text):
    if not isinstance(text, str): return "نامشخص"
    text = text.lower()
    
    # 0. Unknowns
    if any(k in text for k in ['یادم نیست', 'نمیدونم', 'یادم نمیاد']):
        return "نامشخص"

    # 1. Strict (سخت‌گیر)
    if any(k in text for k in ['مهم است', 'اجباری', 'گیر', 'لیست', 'تاثیر مستقیم', 'میخونه']):
        return "سخت‌گیر"
    
    # 2. Free (آزاد)
    # Includes "Positive effect" (Ta'sir Mosbat) because usually that means attendance isn't mandatory
    if any(k in text for k in ['مهم نیست', 'نمی کند', 'ندارد', 'آزاد', 'اختیاری', 'تاثیر مثبت']):
        return "آزاد"
    
    # 3. Moderate (متوسط)
    if any(k in text for k in ['گاهی', 'متوسط']):
        return "متوسط"
        
    return "نامشخص"

# --- EXECUTION ---

# Create/Overwrite the standard columns
df['grading_standard'] = df[GRADING_COL].apply(categorize_grading)
df['attendance_standard'] = df[ATTENDANCE_COL].apply(categorize_attendance)

print("\nFinal Grading Distribution:")
print(df['grading_standard'].value_counts())

print("\nFinal Attendance Distribution:")
print(df['attendance_standard'].value_counts())


Final Grading Distribution:
grading_standard
منصفانه    1687
آسان       1280
سخت‌گیر    1219
نامشخص      362
Name: count, dtype: int64

Final Attendance Distribution:
attendance_standard
سخت‌گیر    2211
آزاد       1918
نامشخص      419
Name: count, dtype: int64


In [33]:
# ---------------------------------------------------------
#                 Save the Clean Data
# ---------------------------------------------------------
from pathlib import Path

# Define output path
output_path = Path(r'../data/processed/clean_data.csv')

# Drop intermediate/raw columns to keep the file clean
# We keep the new 'professor_name_standard', 'grading_standard', etc.
cols_to_drop = [
    'clean_professor_name_raw', 'name_no_title', 
    'clean_course_name', 'grading_status_raw', 'attendance_status_raw'
]

# Create the final dataframe
final_df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Rename standard columns to be the main ones for Phase 3
final_df = final_df.rename(columns={
    'professor_name_standard': 'professor_name',
    'grading_standard': 'grading_status',
    'attendance_standard': 'attendance_status'
})

# Save
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"SUCCESS: Phase 2 Clean data saved to {output_path}")
print(f"Final Row Count: {len(final_df)}")
print(f"Columns: {final_df.columns.tolist()}")

SUCCESS: Phase 2 Clean data saved to ..\data\processed\clean_data.csv
Final Row Count: 4548
Columns: ['id', 'date', 'professor_name_raw', 'department', 'course_name', 'term', 'comment_text', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6', 'clean_department', 'clean_comment_text', 'professor_name', 'grading_status', 'attendance_status']
